In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [15]:
word_list = []
emotion = []

with open('pos_pol_word.txt', encoding = "UTF-8") as pos:
    for idx, line in enumerate(pos):
        if idx > 18:
            word_list.append(line.strip())
            emotion.append('2')
            
with open('neg_pol_word.txt', encoding = "UTF-8") as pos:
    for idx, line in enumerate(pos):
        if idx > 18:
            word_list.append(line.strip())
            emotion.append('0')
            
            
with open('obj_unknown_pol_word.txt', encoding = "UTF-8") as pos:
    for idx, line in enumerate(pos):
        if idx > 19:
            word_list.append(line.strip())
            emotion.append('1')
            
emotion = [int(x) for x in emotion]

In [3]:
emotion_pd = pd.DataFrame(emotion)

In [4]:
#데이터 전처리
# Tokenizer 객체 생성
# num_words: 사용할 단어의 최대 개수 (가장 빈도가 높은 단어부터 사용)
# oov_token: 단어 사전에 없는 단어를 대체할 토큰
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')

# 데이터에 있는 텍스트를 기반으로 단어 사전 생성
tokenizer.fit_on_texts(word_list)

# 텍스트를 단어 사전에 따라 숫자 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(word_list)

# 패딩 처리
# maxlen: 시퀀스의 최대 길이
# padding: 'post'는 시퀀스 뒤에 패딩을 추가, 'pre'는 앞에 추가
# truncating: 'post'는 시퀀스 뒤를 잘라냄, 'pre'는 앞을 잘라냄
padded = pad_sequences(sequences, maxlen=10, padding='post', truncating='post')

In [5]:
# Sequential 모델 생성
model = Sequential([
    # Embedding 레이어: 단어를 고정 크기의 벡터로 임베딩
    # 10000: 단어 사전의 크기
    # 16: 임베딩 벡터의 차원
    # input_length=10: 입력 시퀀스의 길이
    Embedding(10000, 16, input_length=10),

    # LSTM 레이어: 순환 신경망의 한 종류로, 시퀀스 데이터 처리에 적합
    # 32: LSTM 유닛의 개수
    LSTM(32),

    # Dense 레이어: 완전 연결 레이어
    # 24: 뉴런의 개수
    # activation='relu': 활성화 함수로 ReLU 사용
    Dense(24, activation='relu'),

    # Dropout 레이어: 과적합 방지를 위해 일부 뉴런을 무작위로 비활성화
    # 0.5: 비활성화할 뉴런의 비율
    Dropout(0.5),

    # Dense 레이어: 출력 레이어
    # 3: 출력 클래스의 개수 (긍정, 부정, 중립)
    # activation='softmax': 다중 분류를 위한 활성화 함수
    Dense(3, activation='softmax')
])

# 모델 컴파일
# loss='sparse_categorical_crossentropy': 다중 분류 손실 함수
# optimizer='adam': 최적화 알고리즘으로 Adam 사용
# metrics=['accuracy']: 정확도를 평가 지표로 사용
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
#모델 학습
model.fit(padded, emotion_pd, epochs=50)

Epoch 1/50
465/465 [==============================] - 3s 4ms/step - loss: 0.6734 - accuracy: 0.6934
Epoch 2/50
465/465 [==============================] - 2s 5ms/step - loss: 0.2065 - accuracy: 0.9463
Epoch 3/50
465/465 [==============================] - 2s 4ms/step - loss: 0.0930 - accuracy: 0.9782
Epoch 4/50
465/465 [==============================] - 2s 5ms/step - loss: 0.0704 - accuracy: 0.9818
Epoch 5/50
465/465 [==============================] - 2s 4ms/step - loss: 0.0563 - accuracy: 0.9834
Epoch 6/50
465/465 [==============================] - 2s 4ms/step - loss: 0.0465 - accuracy: 0.9837
Epoch 7/50
465/465 [==============================] - 2s 4ms/step - loss: 0.0423 - accuracy: 0.9851
Epoch 8/50
465/465 [==============================] - 2s 5ms/step - loss: 0.0384 - accuracy: 0.9856
Epoch 9/50
465/465 [==============================] - 2s 5ms/step - loss: 0.0336 - accuracy: 0.9859
Epoch 10/50
465/465 [==============================] - 2s 5ms/step - loss: 0.0360 - accuracy: 0.9867

In [10]:
sample_data = ['거북하다', '각별하다', '행복하다', '가치있는']

In [11]:
# 텍스트를 단어 사전에 따라 숫자 시퀀스로 변환
new_sequences = tokenizer.texts_to_sequences(sample_data)

# 패딩 처리
# maxlen: 시퀀스의 최대 길이
# padding: 'post'는 시퀀스 뒤에 패딩을 추가, 'pre'는 앞에 추가
# truncating: 'post'는 시퀀스 뒤를 잘라냄, 'pre'는 앞을 잘라냄
new_padded = pad_sequences(new_sequences, maxlen=10, padding='post', truncating='post')

In [14]:
print(predictions)

[[1.0000000e+00 2.5275311e-36 3.7102641e-09]
 [2.8788704e-11 6.0516418e-18 1.0000000e+00]
 [2.2766038e-05 4.7413682e-08 9.9997723e-01]
 [5.1207597e-11 1.5418079e-17 1.0000000e+00]]


In [12]:
# 예측 수행
#model = load_model('emotion_model.h5')
predictions = model.predict(new_padded)
#new_model = tf.keras.models.load_model('emotion_model.h5')
# 예측 결과 출력
for i, pred in enumerate(predictions):
    predicted_label = np.argmax(pred)
    if predicted_label == 0:
        print(f"'{sample_data[i]}'의 예측 감정: 긍정")
        print()
    elif predicted_label == 1:
        print(f"'{sample_data[i]}'의 예측 감정: 부정")
        print()
    else:
        print(f"'{sample_data[i]}'의 예측 감정: 중립")
        print()

1/1 [==============================] - 0s 23ms/step
'거북하다'의 예측 감정: 긍정

'각별하다'의 예측 감정: 중립

'행복하다'의 예측 감정: 중립

'가치있는'의 예측 감정: 중립

